## PYHTON DB API

## PYODBC

pip install pyodbc

In [1]:
import pyodbc

## SQL SERVER AUTHENTİCATİON

In [2]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER= server, DATABASE = database, UID= user, PWD= password)
        print("Successfull")
    except Error as err:
        print(f"Error:'{err}'")
    return conn

In [3]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client RDA 11.0',
 'SQL Server Native Client 11.0',
 'ODBC Driver 17 for SQL Server']

## WİNDOWS AUTHENTİCATİON

In [5]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [6]:
conn = pyodbc.connect(conn_string);

## CURSOR PROCESS

In [7]:
crs = conn.cursor()

In [8]:
conn.autocommit = True # conn.commit() yazmamak için

## CREATE DATABASE

In [9]:
def create_database(conn, create_database_query): #parametreler: connection nesnesi ve query nesnesi
    crs = conn.cursor() #bir cursor nesnesi oluşturuyoruz. bunu query sonucundaki değerleri yakalamak için kullanacağız.
    try:
        crs.execute(create_database_query)#query'yi çalıştır sonucu yakala. queryleri excecute ile çalıştırıyoruz
        print("Database is Created Successfuly")
    except Error as err:
            print(f"Error:'{err}'")

In [10]:
create_database_query = 'CREATE DATABASE testb'

In [11]:
create_database(conn, create_database_query)

Database is Created Successfuly


In [12]:
query = 'USE testb'

In [15]:
crs.execute(query)

In [16]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [17]:
execute_query(conn, query)

Query Succeessful!


In [18]:
query = 'CREATE TABLE TestTable (\
           ID INT IDENTITY (1,1) NOT NULL, \
           FirstName VARCHAR(255) NOT NULL,\
           LastName VARCHAR(255) NOT NULL,\
           PRIMARY KEY(ID))'

In [19]:
execute_query(conn, query)

Query Succeessful!


## İNSERT

In [24]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [ ]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [23]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Tom', 'Cat')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Jerry', 'Mouse')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Owen', 'William')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)",'Stefan', 'Müller')

## CURSOR FUNCTİON

In [27]:
crs.execute("DELETE FROM TestTable WHERE FirstName = 'jerry'")

## FETCHONE

In [29]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
row = crs.fetchone() # tablodaki ilk kaydı getirir. her çalıştırdıgmızda bir sonraki satırı getirir
row

('Tom', 'Cat')

## FETCHMANY

In [31]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
row = crs.fetchmany(3) 
row

[('Tom', 'Cat'), ('Jerry', 'Mouse'), ('Owen', 'William')]

##FETChall

In [32]:
crs.execute('SELECT FirstName, LastName FROM TestTable')
row = crs.fetchall() 
row

[('Tom', 'Cat'),
 ('Jerry', 'Mouse'),
 ('Owen', 'William'),
 ('Stefan', 'Müller'),
 ('Bob', 'Marley')]

## CREATE DataFrame

In [33]:
import pandas as pd

In [34]:
crs.execute('SELECT * FROM TestTable')
result = crs.fetchall() 
result

[(1, 'Tom', 'Cat'),
 (2, 'Jerry', 'Mouse'),
 (3, 'Owen', 'William'),
 (4, 'Stefan', 'Müller'),
 (5, 'Bob', 'Marley')]

In [36]:
df = pd.DataFrame(result)
df

,0
0,"[1, Tom, Cat]"
1,"[2, Jerry, Mouse]"
2,"[3, Owen, William]"
3,"[4, Stefan, Müller]"
4,"[5, Bob, Marley]"


In [37]:
a = []
for result in result:
    result = list(result)
    a.append(result)

In [38]:
a

[[1, 'Tom', 'Cat'],
 [2, 'Jerry', 'Mouse'],
 [3, 'Owen', 'William'],
 [4, 'Stefan', 'Müller'],
 [5, 'Bob', 'Marley']]

In [40]:
colums = ["ID","FirstName","LastNmae"]
df1 = pd.DataFrame(a, columns=colums)
df1

,ID,FirstName,LastNmae
0,1,Tom,Cat
1,2,Jerry,Mouse
2,3,Owen,William
3,4,Stefan,Müller
4,5,Bob,Marley


In [41]:
query = 'USE SampleRetail'
execute_query(conn, query)

Query Succeessful!


In [42]:
df_prod = pd.read_sql("select * from product.product", con = conn)
df_prod.head()

,product_id,product_name,brand_id,category_id,model_year,list_price
0,1,DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 ...,23,4,2021,23.99
1,2,NS-SP1800BL 5.1-Channel Home Theater System (B...,5,5,2021,136.99
2,3,Acoustimass 6 Series V Home Theater Speaker Sy...,24,5,2021,599.00
3,4,Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3...,6,4,2021,151.99
4,5,Details About Samsung Gear Iconx 2018 Edition ...,1,14,2021,199.99
